# Code for analysis of crispresso editing information

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')
plt.rc('font', family='Helvetica')

In [3]:
def crispresso_compiler(samp, sample_id, folder_path):
    """ 
    Takes in list of sample names (see cell above)
    Returns compiled dictionary of dataframes containing editing information for each sensor
    
    Includes:
    1. corr_perc = pure correct editing
    2. target_base_edit_perc = target base editing perc (including edits with bystander editing)
    3. wt_perc
    4. byproduct information (indels, substitutions, ambiguous)
    """

    df_edits = []
    for k in samp: 
        
        try:
            concated = pd.read_csv(f'{folder_path}/{k}_crispresso_aggregated.csv')

        except:
            concated = pd.read_csv(f'{folder_path}/{k}_crispresso_aggregated_v2.csv')
            
        concated = concated.fillna(0)

        #somehow wasn't able to find the initial set of code that I used for this...

        #go through all of the samples and do it step by step
        sample_ids = []
        #sample_num = []
        rii = []
        raaa = []
        r_lowqual = []
        ra_hdr = []
        ra_wt = []
        r_unaligned = []
        no_edit = []
        correct = []
        target_base_editing = []
        byproduct_all = []
        byproduct_indel = []
        byproduct_sub = []
        byproduct_ambig = []
        for i in np.unique(concated['Guide_ID']):
            sample_ids.append(i)

            subset = concated[concated['Guide_ID']==i]

            wt = subset[subset['Amplicon']=='Reference']
            edit = subset[subset['Amplicon']=='HDR']

            #sample_num.append(wt['sample_num'].values[0])
            rii1 = wt['Reads_in_input'].values[0]
            r1 = wt['Reads_aligned_all_amplicons'].values[0]
            rii.append(rii1)
            raaa.append(r1)
            r_lowqual.append(rii1-r1)

            r2 = edit['Reads_aligned'].values[0]
            r3 = wt['Reads_aligned'].values[0]
            ra_hdr.append(r2)
            ra_wt.append(r3)
            r_unaligned.append(r1 - (r2+r3))

            no_edit.append(wt['Unmodified'].values[0])
            correct.append(edit['Unmodified'].values[0])
            target_base_editing.append(edit['Modified'].values[0] + edit['Unmodified'].values[0])

            byprod_all = wt['Modified'].values[0] + edit['Modified'].values[0] + (r1 - (r2+r3)) #add unaligned reads
            sub_all = wt['Only Substitutions'].values[0] + edit['Only Substitutions'].values[0]
            indel_all = wt['Only Deletions'].values[0] + wt['Only Insertions'].values[0] + wt['Insertions and Deletions'].values[0] + edit['Only Deletions'].values[0] + edit['Only Insertions'].values[0] + edit['Insertions and Deletions'].values[0]
            ambig_all = byprod_all - sub_all - indel_all

            byproduct_all.append(byprod_all)
            byproduct_indel.append(indel_all)
            byproduct_sub.append(sub_all)
            byproduct_ambig.append(ambig_all)


        cols = ["Guide_ID", "Reads_in_input","Reads_lowqual", "Reads_aligned_all_amplicons","Reads_aligned_WT", "Reads_aligned_HDR", "Reads_unaligned", "WT","correct_edit", "target_base_edit", "byproduct_all","byproduct_INDEL","byproduct_sub","byproduct_ambiguous"]
        col_vals = [ sample_ids, rii, r_lowqual, raaa, ra_wt, ra_hdr, r_unaligned, no_edit, correct, target_base_editing, byproduct_all, byproduct_indel, byproduct_sub, byproduct_ambig]

        out = pd.DataFrame(dict(zip(cols, col_vals)))
        out['corr_perc'] = 100*(out['correct_edit']/out['Reads_aligned_all_amplicons'])
        out['target_base_edit_perc'] = 100*(out['target_base_edit']/out['Reads_aligned_all_amplicons'])
        out['WT_perc'] = 100*(out['WT']/out['Reads_aligned_all_amplicons'])
        out['byproduct_all_perc'] =  100*(out['byproduct_all']/out['Reads_aligned_all_amplicons'])
        out['byproduct_INDEL_perc'] =  100*(out['byproduct_INDEL']/out['Reads_aligned_all_amplicons'])
        out['byproduct_sub_perc'] =  100*(out['byproduct_sub']/out['Reads_aligned_all_amplicons'])
        out['byproduct_ambiguous_perc'] =100*(out['byproduct_ambiguous']/out['Reads_aligned_all_amplicons'])
        out = out.fillna(0)

        df_edits.append(out)

    edit_dict = dict(zip(sample_id, df_edits))

    return edit_dict

In [4]:
def MLE_merge(edit_dict, samps_to_merge, new_name):
    """ 
    combine replicates to generate best estimate of sensor editing
    only works for combinations of 3 replicates (could be modified)

    MODIFIED TO WORK FOR MULTIPLE REPLICATES
    """
    cols_to_add = ['Reads_in_input', 'Reads_lowqual',
       'Reads_aligned_all_amplicons', 'Reads_aligned_WT', 'Reads_aligned_HDR',
       'Reads_unaligned', 'WT', 'correct_edit', 'target_base_edit',
       'byproduct_all', 'byproduct_INDEL', 'byproduct_sub',
       'byproduct_ambiguous',]
    
    comb_holder = []
    for k in samps_to_merge:
        
        a = edit_dict[k[0]]
        comb = a[cols_to_add]

        for jj in k[1:]:
            print(jj)
            j_df = edit_dict[jj]
            comb+=j_df[cols_to_add]
        
        #a = edit_dict[k[0]]
        #b = edit_dict[k[1]]
        #c = edit_dict[k[2]]

        #check that guide indeces match
        #assert list(a['Guide_ID'])==list(b['Guide_ID'])==list(c['Guide_ID']), 'indexes dont match'

        #a2 = a[cols_to_add]
        #b2 = b[cols_to_add]
        #c2 = c[cols_to_add]

        #comb = a2+b2+c2

        comb['Guide_ID'] = a['Guide_ID']

        #and reorganize columns
        col_order = ['Guide_ID'] + cols_to_add
        out = comb[col_order]

        #and then calcualte percentages
        out['corr_perc'] = 100*(out['correct_edit']/out['Reads_aligned_all_amplicons'])
        out['target_base_edit_perc'] = 100*(out['target_base_edit']/out['Reads_aligned_all_amplicons'])
        out['WT_perc'] = 100*(out['WT']/out['Reads_aligned_all_amplicons'])
        out['byproduct_all_perc'] =  100*(out['byproduct_all']/out['Reads_aligned_all_amplicons'])
        out['byproduct_INDEL_perc'] =  100*(out['byproduct_INDEL']/out['Reads_aligned_all_amplicons'])
        out['byproduct_sub_perc'] =  100*(out['byproduct_sub']/out['Reads_aligned_all_amplicons'])
        out['byproduct_ambiguous_perc'] =100*(out['byproduct_ambiguous']/out['Reads_aligned_all_amplicons'])
        out = out.fillna(0)

        comb_holder.append(out)

    out_dict = dict(zip(new_name, comb_holder))

    return out_dict

In [5]:
samp_CBE = ['D24-12799-6840R_guide_split_BARCODES',
 'D24-12800-6840R_guide_split_BARCODES',
 'D24-12801-6840R_guide_split_BARCODES',
 'D24-12802-6840R_guide_split_BARCODES',
 'D24-12803-6840R_guide_split_BARCODES',
 'D24-12804-6840R_guide_split_BARCODES',
 'D24-12805-6840R_guide_split_BARCODES',
 'D24-12806-6840R_guide_split_BARCODES',
 'D24-12807-6840R_guide_split_BARCODES',
 'D24-12808-6840R_guide_split_BARCODES',
 'D24-12809-6840R_guide_split_BARCODES',
 'D24-12810-6840R_guide_split_BARCODES',
 'D24-12811-6840R_guide_split_BARCODES',
 'D24-12812-6840R_guide_split_BARCODES',
 'D24-12813-6840R_guide_split_BARCODES',
 'D24-12814-6840R_guide_split_BARCODES',
 'D24-12815-6840R_guide_split_BARCODES',
 'D24-12816-6840R_guide_split_BARCODES',
 'D24-12817-6840R_guide_split_BARCODES',
 'D24-12818-6840R_guide_split_BARCODES',
 'D24-12819-6840R_guide_split_BARCODES',
 'D24-12820-6840R_guide_split_BARCODES',
 'D24-12821-6840R_guide_split_BARCODES',
 'D24-12822-6840R_guide_split_BARCODES',
 'D24-12823-6840R_guide_split_BARCODES',
 'D24-12824-6840R_guide_split_BARCODES',
 'D24-12825-6840R_guide_split_BARCODES',
 'D24-12826-6840R_guide_split_BARCODES',
 'D24-12827-6840R_guide_split_BARCODES',
 'D24-12828-6840R_guide_split_BARCODES',
 'D24-12829-6840R_guide_split_BARCODES',
 'D24-12830-6840R_guide_split_BARCODES',
 'D24-12831-6840R_guide_split_BARCODES',
 'D24-12832-6840R_guide_split_BARCODES',
 'D24-12833-6840R_guide_split_BARCODES',
 'D24-12834-6840R_guide_split_BARCODES',
 'D24-12835-6840R_guide_split_BARCODES',
 'D24-12836-6840R_guide_split_BARCODES',
 'D24-12837-6840R_guide_split_BARCODES']

sample_id_CBE = ['spleen1', 'spleen2','spleen3','spleen4','spleen5', 'spleen6', 'spleen7', 'spleen8', 'spleen9', 'bonemarrow1','bonemarrow2','bonemarrow3',
            'bonemarrow4','bonemarrow5', 'bonemarrow6','bonemarrow7','bonemarrow8', 'bonemarrow9',
             'bonemarrow10','meninges1','meninges2','meninges3','meninges4','meninges5', 'meninges6', 'meninges7',
             'meninges8', 'meninges9', 'meninges10',
            'input_rep1', 'input_rep2', 'input_rep3', 'd5_rep1', 'd5_rep2', 'd5_rep3', 'd15_rep1',
            'd15_rep2', 'd15_rep3', 'plasmidlib']


samp_ABE = ['D24-12774-6848T_guide_split_BARCODES',
 'D24-12775-6848T_guide_split_BARCODES',
 'D24-12776-6848T_guide_split_BARCODES',
 'D24-12777-6848T_guide_split_BARCODES',
 'D24-12778-6848T_guide_split_BARCODES',
 'D24-12779-6848T_guide_split_BARCODES',
 'D24-12780-6848T_guide_split_BARCODES',
 'D24-12781-6848T_guide_split_BARCODES',
 'D24-12782-6848T_guide_split_BARCODES',
 'D24-12783-6848T_guide_split_BARCODES',
 'D24-12784-6848T_guide_split_BARCODES',
 'D24-12785-6848T_guide_split_BARCODES',
 'D24-12786-6848T_guide_split_BARCODES',
 'D24-12787-6848T_guide_split_BARCODES',
 'D24-12788-6848T_guide_split_BARCODES',
 'D24-12789-6848T_guide_split_BARCODES',
 'D24-12790-6848T_guide_split_BARCODES',
 'D24-12791-6848T_guide_split_BARCODES',
 'D24-12792-6848T_guide_split_BARCODES',
 'D24-12793-6848T_guide_split_BARCODES',
 'D24-12794-6848T_guide_split_BARCODES',
 'D24-12795-6848T_guide_split_BARCODES',
 'D24-12796-6848T_guide_split_BARCODES',
 'D24-12797-6848T_guide_split_BARCODES',
 'D24-12798-6848T_guide_split_BARCODES',]

sample_id_ABE = ['spleen1', 'spleen2','spleen3','spleen4','spleen5', 'bonemarrow1','bonemarrow2','bonemarrow3',
            'bonemarrow4','bonemarrow5','meninges1','meninges2','meninges3','meninges4','meninges5',
            'input_rep1', 'input_rep2', 'input_rep3', 'd5_rep1', 'd5_rep2', 'd5_rep3', 'd15_rep1',
            'd15_rep2', 'd15_rep3', 'plasmidlib']


ABE_samples = pd.DataFrame(dict(zip(['file_name', 'sample'], [samp_ABE, sample_id_ABE])))
CBE_samples = pd.DataFrame(dict(zip(['file_name', 'sample'], [samp_CBE, sample_id_CBE])))

ABE_samp_dict = dict(zip(samp_ABE, sample_id_ABE))
CBE_samp_dict = dict(zip(samp_CBE, sample_id_CBE))

# ABE quantification and MLE generation

In [ ]:
samp = list(ABE_samp_dict.keys())
sample_id = list(ABE_samp_dict.values())
folder_path = '240807Hem_ABE/crispresso'
ABE_edit_dict = crispresso_compiler(samp, sample_id, folder_path)

In [ ]:
samps_to_merge = [['spleen1',
 'spleen2',
 'spleen3',
 'spleen4',
 'spleen5',],
 ['bonemarrow1',
 'bonemarrow2',
 'bonemarrow3',
 'bonemarrow4',
 'bonemarrow5',],
 ['meninges1',
 'meninges2',
 'meninges3',
 'meninges4',
 'meninges5',],
 ['input_rep1',
 'input_rep2',
 'input_rep3',],
 ['d5_rep1',
 'd5_rep2',
 'd5_rep3',],
 ['d15_rep1',
 'd15_rep2',
 'd15_rep3',],
 ['plasmidlib']]

new_name = ['spleen', 'bonemarrow', 'meninges', 'input', 'd5', 'd15', 'plasmid']
out_MLE = MLE_merge(ABE_edit_dict, samps_to_merge, new_name)

# CBE quantification and MLE generation

In [ ]:
samp = list(CBE_samp_dict.keys())
sample_id = list(CBE_samp_dict.values())
folder_path = '240807HemA_CBE/crispresso'
CBE_edit_dict = crispresso_compiler(samp, sample_id, folder_path)

In [ ]:
#and then run it for the legacy guides that were quantified later
samp = list(CBE_samp_dict.keys())
sample_id = list(CBE_samp_dict.values())
folder_path = '240807HemA_CBE/crispresso_legacy'
CBE_edit_dict_legacy = crispresso_compiler(samp, sample_id, folder_path)

In [ ]:
#and updating the results so that the legacy guides have editing quantified
sample_id = list(CBE_samp_dict.values())

new_dfs = []
for sample_considered in sample_id:
    a = CBE_edit_dict_legacy[sample_considered]

    b = CBE_edit_dict[sample_considered]

    b = b[~b['Guide_ID'].isin(list(a['Guide_ID']))]

    new = pd.concat((b,a)).sort_values(by='Guide_ID')
    new_dfs.append(new)


CBE_edit_dict_new = dict(zip(sample_id, new_dfs))

In [ ]:
samps_to_merge = [['spleen1',
 'spleen2',
 'spleen3',
 'spleen4',
 'spleen5',
 'spleen6',
 'spleen7',
 'spleen8',
 'spleen9',],
 ['bonemarrow1',
 'bonemarrow2',
 'bonemarrow3',
 'bonemarrow4',
 'bonemarrow5',
 'bonemarrow6',
 'bonemarrow7',
 'bonemarrow8',
 'bonemarrow9',
 'bonemarrow10',],
 ['meninges1',
 'meninges2',
 'meninges3',
 'meninges4',
 'meninges5',
 'meninges6',
 'meninges7',
 'meninges8',
 'meninges9',
 'meninges10',],
 ['input_rep1',
 'input_rep2',
 'input_rep3',],
 ['d5_rep1',
 'd5_rep2',
 'd5_rep3',],
 ['d15_rep1',
 'd15_rep2',
 'd15_rep3',],
 ['plasmidlib']]

new_name = ['spleen', 'bonemarrow', 'meninges', 'input', 'd5', 'd15', 'plasmid']
out_MLE_CBE = MLE_merge(CBE_edit_dict_new, samps_to_merge, new_name)